In [4]:
# Import libraries
import lxml
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])
        
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')


# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)



Recent News Headlines for AAPL: 
Another China Lockdown Threatens This Apple Product ( Apr-16-22 06:43PM )
Traders Can Clench the Power of Compounding Returns. Here's How ( 10:00AM )
2 Charts That Show Why It's Time to Buy the Dip in Meta Platforms' Stock ( 07:00AM )


Recent News Headlines for TSLA: 
Elon Musk Has an Original Strategy to Take Control of Twitter ( Apr-17-22 10:01AM )
Tesla Stock Vs. BYD Stock: Tesla Shanghai Set To Restart As Earnings Loom, But BYD Set To Seize EV Crown ( 08:55AM )
Dow Jones Futures: Market Rally Weak, But Bulls Still Rule Here; Tesla Shanghai Reopening ( 08:53AM )


Recent News Headlines for AMZN: 
These Upcoming Stock Splits Are Screaming Buys ( Apr-17-22 07:00AM )
Amazon.com Partners With Everyone But SpaceX ( 06:06AM )
3 Warren Buffett Stocks That Can Soar 33% to 60%, According to Wall Street ( 05:57AM )


              Date     Time    neg    neu   pos  compound
Ticker                                                   
AAPL    2022-04-16  06:43P

In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\hriti\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import nltk
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\hriti\AppData\Roaming\nltk_data...


True